In [ ]:
import os
from pyprojroot import here
os.environ["HF_HOME"] = str(here("cache/huggingface"))
import lightning as L
from importlib import reload
import sys
sys.path.append(str(here()))
from models import model_definitions as MD
import torch
import pandas as pd
reload(MD)
torch.set_float32_matmul_precision('medium')
data_module_3 = MD.GoEmotionsDataModule(pd.read_parquet(here("data/goemotions_3.parquet")))
data_module_7 = MD.GoEmotionsDataModule(pd.read_parquet(here("data/goemotions_7.parquet")))
data_module_28 = MD.GoEmotionsDataModule(pd.read_parquet(here("data/goemotions_28.parquet")))

for module in [data_module_3, data_module_7, data_module_28]:
  module.prepare_data()
  module.setup()

test_df_3 = data_module_3.test_df
test_df_7 = data_module_7.test_df
test_df_28 = data_module_28.test_df

trainer = L.Trainer(
  default_root_dir=here("cache/lightning"),
  enable_checkpointing=True,
  fast_dev_run=False,
  accelerator="cpu", 
)

trainer

Seed set to 1
Seed set to 1
Seed set to 1
Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
c:\Users\Plancha\emotion-recognition\.pixi\envs\default\Lib\site-packages\lightning\pytorch\trainer\connectors\logger_connector\logger_connector.py:76: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `lightning.pytorch` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default


In [90]:
pd.set_option("display.max_colwidth", 100)
pd.set_option('display.max_rows',20)
top20 = test_df_28[["text", "emotions"]].head(20)
top20

,text,emotions
42351,*Hey just noticed..* it's your **1st Cakeday** aerosvision! ^(hug),"[[love, surprise], [excitement], [joy], [realization], [excitement, joy, realization]]"
9960,Tell me more about this policy. They still fire here for defending yourself.,"[[unclear], [unclear], [fear, nervousness]]"
12702,Thank you for being the voice of reason in the calamity that is Walmart.,"[[gratitude], [gratitude], [gratitude]]"
40937,"It reads like they -told- him the right address, not fixed it in the app.","[[unclear], [unclear], [unclear]]"
50495,You may be harder on yourself than necessary. Keep praying and confessing your sins. No one is p...,"[[caring], [caring], [anger]]"
43717,I was thinking the same and was relieved when I read peed on kitchen floor!,"[[realization, relief], [relief], [relief]]"
38461,Realizing that thirty isn't old at all.,"[[realization], [realization], [realization]]"
47230,I drive an old car but it is clean and well maintained.,"[[love], [approval], [approval]]"
29922,I imagine it would be helpful for little kids who cant reach the faucet.,"[[unclear], [optimism], [optimism]]"
52912,"Oh, the Custom is a great guitar. But it doesn't have same snarl of the Epi.","[[admiration, disappointment], [admiration], [disappointment]]"


In [78]:
reload(MD)
distil_bert_wmse_28 = MD.DistilBertFinetuneOnWeightedMSE.load_from_checkpoint(
  here(r"models\model_checkpoints\distil_bert_onWMSE_28_12epchs.ckpt"),
  n_emotions=28
)
distil_bert_wmse_28

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


DistilBertFinetuneOnWeightedMSE(
  (model): DistilBertForSequenceClassification(
    (distilbert): DistilBertModel(
      (embeddings): Embeddings(
        (word_embeddings): Embedding(30522, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (transformer): Transformer(
        (layer): ModuleList(
          (0-5): 6 x TransformerBlock(
            (attention): DistilBertSdpaAttention(
              (dropout): Dropout(p=0.1, inplace=False)
              (q_lin): Linear(in_features=768, out_features=768, bias=True)
              (k_lin): Linear(in_features=768, out_features=768, bias=True)
              (v_lin): Linear(in_features=768, out_features=768, bias=True)
              (out_lin): Linear(in_features=768, out_features=768, bias=True)
            )
            (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=T

In [88]:
import numpy as np
def out_to_emotion(out, df):
  """Print model outputs to a df"""
  # out is a list of lists of probabilities for each emotion
  ret = pd.DataFrame(
    np.array([i.numpy() for i in out]).reshape(-1, 28),
    # columns where they start with emotion_
    columns = df.columns[df.columns.str.startswith("emotion_")],
    index = df.index
  )
  # add the text column and put it first
  return ret
out = trainer.predict(
  distil_bert_wmse_28,
  test_df_28.head(20)["text"].tolist()
)
outs = out_to_emotion(out, test_df_28.head(20))
outs

Predicting: |          | 0/? [00:00<?, ?it/s]

,emotion_sadness,emotion_unclear,emotion_love,emotion_gratitude,emotion_disapproval,emotion_amusement,emotion_disappointment,emotion_disgust,emotion_admiration,emotion_realization,...,emotion_approval,emotion_nervousness,emotion_embarrassment,emotion_surprise,emotion_anger,emotion_grief,emotion_pride,emotion_desire,emotion_relief,emotion_fear
42351,0.021409,0.118298,0.115330,0.010858,0.001586,0.028794,0.011490,0.000493,0.090214,0.020286,...,0.016757,0.001934,0.002436,0.029705,0.000424,0.001044,0.009969,0.020602,0.002497,0.000222
9960,0.028683,0.570094,0.018833,0.001120,0.120346,0.006267,0.028092,0.013842,0.019691,0.093136,...,0.165855,0.004121,0.007583,0.002573,0.190452,0.002483,0.015137,0.021172,0.002894,0.021566
12702,0.004395,0.006869,0.006685,0.999054,0.003864,0.002593,0.006024,0.003706,0.208734,0.005213,...,0.037500,0.000281,0.000182,0.003818,0.020480,0.003643,0.012881,0.012471,0.003149,0.001429
40937,0.004407,0.680003,0.007604,0.004316,0.125643,0.006490,0.017550,0.001694,0.013223,0.035425,...,0.138190,0.000357,0.000474,0.007757,0.006907,0.000604,0.000953,0.010158,0.000647,0.000285
50495,0.034699,0.164577,0.013336,0.002230,0.102685,0.000216,0.032917,0.103990,0.056395,0.031263,...,0.237150,0.000929,0.004883,0.000738,0.052335,0.000718,0.001016,0.005201,0.003338,0.024159
43717,0.033376,0.087935,0.002550,0.024184,0.002704,0.220442,0.030382,0.012184,0.013453,0.336302,...,0.065237,0.009147,0.042507,0.216927,0.000733,0.001487,0.002873,0.002958,0.416690,0.001518
38461,0.015738,0.519117,0.000028,0.000259,0.086866,0.001709,0.060877,0.009589,0.006810,0.453710,...,0.056549,0.003588,0.004208,0.164319,0.010164,0.000462,0.007325,0.006068,0.011102,0.000752
47230,0.059516,0.307622,0.005548,0.001023,0.067332,0.000846,0.039613,0.025754,0.199462,0.035983,...,0.512995,0.001785,0.000637,0.000114,0.004052,0.003851,0.002206,0.006555,0.010888,0.000285
29922,0.011155,0.222883,0.021757,0.009750,0.017003,0.110490,0.012918,0.004937,0.328358,0.047797,...,0.174832,0.006292,0.001365,0.041671,0.000227,0.000731,0.000365,0.062970,0.015780,0.008609
52912,0.005481,0.233915,0.023765,0.001989,0.144038,0.000233,0.039018,0.019268,0.227504,0.024339,...,0.139201,0.000295,0.001341,0.019074,0.011946,0.000541,0.000136,0.000403,0.000022,0.003748


In [91]:
# join top20 with outs
top20_p = top20.join(outs)
top20_p

,text,emotions,emotion_sadness,emotion_unclear,emotion_love,emotion_gratitude,emotion_disapproval,emotion_amusement,emotion_disappointment,emotion_disgust,...,emotion_approval,emotion_nervousness,emotion_embarrassment,emotion_surprise,emotion_anger,emotion_grief,emotion_pride,emotion_desire,emotion_relief,emotion_fear
42351,*Hey just noticed..* it's your **1st Cakeday** aerosvision! ^(hug),"[[love, surprise], [excitement], [joy], [realization], [excitement, joy, realization]]",0.021409,0.118298,0.115330,0.010858,0.001586,0.028794,0.011490,0.000493,...,0.016757,0.001934,0.002436,0.029705,0.000424,0.001044,0.009969,0.020602,0.002497,0.000222
9960,Tell me more about this policy. They still fire here for defending yourself.,"[[unclear], [unclear], [fear, nervousness]]",0.028683,0.570094,0.018833,0.001120,0.120346,0.006267,0.028092,0.013842,...,0.165855,0.004121,0.007583,0.002573,0.190452,0.002483,0.015137,0.021172,0.002894,0.021566
12702,Thank you for being the voice of reason in the calamity that is Walmart.,"[[gratitude], [gratitude], [gratitude]]",0.004395,0.006869,0.006685,0.999054,0.003864,0.002593,0.006024,0.003706,...,0.037500,0.000281,0.000182,0.003818,0.020480,0.003643,0.012881,0.012471,0.003149,0.001429
40937,"It reads like they -told- him the right address, not fixed it in the app.","[[unclear], [unclear], [unclear]]",0.004407,0.680003,0.007604,0.004316,0.125643,0.006490,0.017550,0.001694,...,0.138190,0.000357,0.000474,0.007757,0.006907,0.000604,0.000953,0.010158,0.000647,0.000285
50495,You may be harder on yourself than necessary. Keep praying and confessing your sins. No one is p...,"[[caring], [caring], [anger]]",0.034699,0.164577,0.013336,0.002230,0.102685,0.000216,0.032917,0.103990,...,0.237150,0.000929,0.004883,0.000738,0.052335,0.000718,0.001016,0.005201,0.003338,0.024159
43717,I was thinking the same and was relieved when I read peed on kitchen floor!,"[[realization, relief], [relief], [relief]]",0.033376,0.087935,0.002550,0.024184,0.002704,0.220442,0.030382,0.012184,...,0.065237,0.009147,0.042507,0.216927,0.000733,0.001487,0.002873,0.002958,0.416690,0.001518
38461,Realizing that thirty isn't old at all.,"[[realization], [realization], [realization]]",0.015738,0.519117,0.000028,0.000259,0.086866,0.001709,0.060877,0.009589,...,0.056549,0.003588,0.004208,0.164319,0.010164,0.000462,0.007325,0.006068,0.011102,0.000752
47230,I drive an old car but it is clean and well maintained.,"[[love], [approval], [approval]]",0.059516,0.307622,0.005548,0.001023,0.067332,0.000846,0.039613,0.025754,...,0.512995,0.001785,0.000637,0.000114,0.004052,0.003851,0.002206,0.006555,0.010888,0.000285
29922,I imagine it would be helpful for little kids who cant reach the faucet.,"[[unclear], [optimism], [optimism]]",0.011155,0.222883,0.021757,0.009750,0.017003,0.110490,0.012918,0.004937,...,0.174832,0.006292,0.001365,0.041671,0.000227,0.000731,0.000365,0.062970,0.015780,0.008609
52912,"Oh, the Custom is a great guitar. But it doesn't have same snarl of the Epi.","[[admiration, disappointment], [admiration], [disappointment]]",0.005481,0.233915,0.023765,0.001989,0.144038,0.000233,0.039018,0.019268,...,0.139201,0.000295,0.001341,0.019074,0.011946,0.000541,0.000136,0.000403,0.000022,0.003748


In [105]:
# per text, print most probable emotions and real emotions
for i, row in top20_p.iterrows():
  print(f"Text: {row['text']}")
  print("Predicted emotions:")
  emotions = row[2:].sort_values(ascending=False).head(3)
  for emotion, prob in emotions.items():
    print(f"{emotion}: {prob:.4f}")
  print("Real emotions:")
  print(str([i.tolist() for i in row['emotions']]))
  print("\n")

Text: *Hey just noticed..* it's your **1st Cakeday** aerosvision! ^(hug)
Predicted emotions:
emotion_joy: 0.3933
emotion_excitement: 0.3343
emotion_unclear: 0.1183
Real emotions:
[['love', 'surprise'], ['excitement'], ['joy'], ['realization'], ['excitement', 'joy', 'realization']]


Text: Tell me more about this policy. They still fire here for defending yourself.
Predicted emotions:
emotion_unclear: 0.5701
emotion_caring: 0.2887
emotion_anger: 0.1905
Real emotions:
[['unclear'], ['unclear'], ['fear', 'nervousness']]


Text: Thank you for being the voice of reason in the calamity that is Walmart.
Predicted emotions:
emotion_gratitude: 0.9991
emotion_admiration: 0.2087
emotion_approval: 0.0375
Real emotions:
[['gratitude'], ['gratitude'], ['gratitude']]


Text: It reads like they -told- him the right address, not fixed it in the app.
Predicted emotions:
emotion_unclear: 0.6800
emotion_approval: 0.1382
emotion_disapproval: 0.1256
Real emotions:
[['unclear'], ['unclear'], ['unclear']]


T

In [104]:
str([i.tolist() for i in row['emotions']])

"[['love', 'surprise'], ['excitement'], ['joy'], ['realization'], ['excitement', 'joy', 'realization']]"